In [2]:
import qutip as qt
import numpy as np
print(qt.__version__)


5.0.4


## First we define state and density matrix

In [3]:

def create_w_state(n):
    basis_states = [qt.basis([2]*n, [1 if i == j else 0 for i in range(n)]) for j in range(n)]
    w_state = sum(basis_states) / np.sqrt(n)
    return w_state

def compute_density_matrix(state):
    return state * state.dag()

num_qubits = 3 
w_state = create_w_state(num_qubits)
p_W = compute_density_matrix(w_state)

print("W state:")
print(w_state)
np.set_printoptions(linewidth=200, precision=3, suppress=True)
print("Density Matrix in Matrix Form:\n")
print(np.real(p_W.full()))


W state:
Quantum object: dims=[[2, 2, 2], [1, 1, 1]], shape=(8, 1), type='ket', dtype=Dense
Qobj data =
[[0.        ]
 [0.57735027]
 [0.57735027]
 [0.        ]
 [0.57735027]
 [0.        ]
 [0.        ]
 [0.        ]]
Density Matrix in Matrix Form:

[[0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.333 0.333 0.    0.333 0.    0.    0.   ]
 [0.    0.333 0.333 0.    0.333 0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.333 0.333 0.    0.333 0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.   ]]


## Conditional Mutual Information bound check

In [4]:
def compute_reduced_density_matrices(p_W):
    p_A = p_W.ptrace([0])  # Tracing out B and C
    p_B = p_W.ptrace([1])  # Tracing out A and C
    p_C = p_W.ptrace([2])  # Tracing out A and B
    p_AB = p_W.ptrace([0, 1])  # Tracing out C
    p_AC = p_W.ptrace([0, 2])  # Tracing out B
    p_BC = p_W.ptrace([1, 2])  # Tracing out A
    return p_A, p_B, p_C, p_AB, p_AC, p_BC


p_A, p_B, p_C, p_AB, p_AC, p_BC = compute_reduced_density_matrices(p_W)

def compute_entropies(p_A, p_B, p_C, p_AB, p_AC, p_BC, p_W):
    S_A = qt.entropy_vn(p_A)
    S_B = qt.entropy_vn(p_B)
    S_C = qt.entropy_vn(p_C)
    S_AB = qt.entropy_vn(p_AB)
    S_AC = qt.entropy_vn(p_AC)
    S_BC = qt.entropy_vn(p_BC)
    S_W = qt.entropy_vn(p_W)  # Total system entropy
    return S_A, S_B, S_C, S_AB, S_AC, S_BC, S_W

S_A, S_B, S_C, S_AB, S_AC, S_BC, S_W = compute_entropies(p_A, p_B, p_C, p_AB, p_AC, p_BC, p_W)


In [5]:
# Nicely formatted printout
entropy_labels = ["S(ρ_A)", "S(ρ_B)", "S(ρ_C)", "S(ρ_AB)", "S(ρ_AC)", "S(ρ_BC)", "S(ρ_W)"]
entropy_values = [S_A, S_B, S_C, S_AB, S_AC, S_BC, S_W]

# Print header
print("=" * 40)
print(f"{'Subsystem':<10} | {'Entropy':>12}")
print("=" * 40)

# Print entropy values
for label, value in zip(entropy_labels, entropy_values):
    print(f"{label:<10} | {value:12.6f}")

print("=" * 40)


Subsystem  |      Entropy
S(ρ_A)     |     0.636514
S(ρ_B)     |     0.636514
S(ρ_C)     |     0.636514
S(ρ_AB)    |     0.636514
S(ρ_AC)    |     0.636514
S(ρ_BC)    |     0.636514
S(ρ_W)     |    -0.000000


## Compute the Conditional Mutual Information

In [6]:
I_ABcC = S_AC + S_BC - S_W - S_C
print("Conditional mutual information I(A:B|C) =",I_ABcC)

Conditional mutual information I(A:B|C) = 0.636514168294817


## Compute 3 possible bounds using Z operator

In [10]:
Z = qt.sigmaz()
Z_2 = qt.tensor(Z, Z)
Z_full = qt.tensor(Z, Z, Z)


avg_Z_A = (p_A * Z).tr()
avg_Z_B = (p_B * Z).tr()
avg_Z_C = (p_C * Z).tr()
avg_Z_AC = (p_AC * Z_2).tr()
avg_Z_BC = (p_BC * Z_2).tr()
avg_Z_AB = (p_AB * Z_2).tr()
avg_Z_full = (p_W * Z_full).tr()

Z_norm = 1



In [11]:
# Define labels and values
z_labels = [
    "⟨Z_A⟩", "⟨Z_B⟩", "⟨Z_C⟩",
    "⟨Z_AC⟩", "⟨Z_BC⟩", "⟨Z_AB⟩",
    "⟨Z_W⟩ (Full)"
]
z_values = [
    avg_Z_A, avg_Z_B, avg_Z_C,
    avg_Z_AC, avg_Z_BC, avg_Z_AB,
    avg_Z_full
]

# Print header
print("=" * 40)
print(f"{'Observable':<10} | {'Expectation Value':>20}")
print("=" * 40)

# Print expectation values
for label, value in zip(z_labels, z_values):
    print(f"{label:<10} | {value:20.6f}")

print("=" * 40)


Observable |    Expectation Value
⟨Z_A⟩      |             0.333333
⟨Z_B⟩      |             0.333333
⟨Z_C⟩      |             0.333333
⟨Z_AC⟩     |            -0.333333
⟨Z_BC⟩     |            -0.333333
⟨Z_AB⟩     |            -0.333333
⟨Z_W⟩ (Full) |            -1.000000


In [15]:
f_1 = 0.5*(((avg_Z_full-(avg_Z_A*avg_Z_BC))**2 + (avg_Z_full-(avg_Z_B*avg_Z_AC))**2)/(2*3*Z_norm**2)-2)
f_2 = (avg_Z_full-avg_Z_A*avg_Z_B*avg_Z_C)**2/(2*3*Z_norm**2)-2
f_3 = (avg_Z_full-avg_Z_A*avg_Z_B)**2/(2*2*Z_norm**2) 
f_4 = (avg_Z_full*avg_Z_C - avg_Z_AC*avg_Z_BC)**2/(2*4*Z_norm**2)

print(" f_1 = ", np.real(f_1),"\n f_2 = ", np.real(f_2),"\n f_3 =",np.real(f_3),"\n f_4 = ", np.real(f_4))

 f_1 =  -0.868312757201646 
 f_2 =  -1.8207590306355739 
 f_3 = 0.3086419753086421 
 f_4 =  0.02469135802469138
